In [1]:
!pip install transformers torch protobuf tiktoken sentencepiece bitsandbytes==0.43.3 accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 9.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 31.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 KB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 56.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 70.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.3 MB/s

In [6]:
!pip install bitsandbytes -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 52.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.3
    Uninstalling bitsandbytes-0.43.3:
      Successfully uninstalled bitsandbytes-0.43.3


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math

In [2]:
model_name = "andreidima/Llama-2-7b-Romanian-qlora"

tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='cuda:0')
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='cuda:0')

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096

In [4]:
def compute_metrics(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    input_ids = inputs["input_ids"]
    
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    
    neg_log_likelihood = outputs.loss
    perplexity = math.exp(neg_log_likelihood.item())
    
    return [perplexity, neg_log_likelihood.item()]

In [5]:
import json
from tqdm import tqdm

In [6]:
regions = [
    'Ardeal',
    'Banat',
    'Bucovina',
    'Canada_EN',
    'Canada_Quebec',
    'Crisana',
    'Dobrogea',
    'Germania',
    'Italia',
    'Maramures',
    'Moldova',
    'Muntenia',
    'Oltenia',
    'Serbia',
    'Spania',
    'Ucraina',
    'UK'
]

In [10]:
import json
from tqdm import tqdm
import numpy as np

In [11]:
results_regions = {}
for region in regions:
    print(region)
    results_regions[region] = []
    results_content = {"perplexity": [], "neg_log_likelihood": []}
    results_titles = {"perplexity": [], "neg_log_likelihood": []}

    with open(f"dataset/{region}.json") as f:
        region_json = json.load(f)

    for row in tqdm(region_json):
        row_cnt_result = compute_metrics(model, tokenizer, row['content']  if 'content' in row else row['text'] )
        # print(row_cnt_result)
        results_content['perplexity'].append(row_cnt_result[0])
        results_content['neg_log_likelihood'].append(row_cnt_result[1])
        
        row_title_result = compute_metrics(model, tokenizer, row['title'] )
        results_titles['perplexity'].append(row_title_result[0])
        results_titles['neg_log_likelihood'].append(row_title_result[1])

    perp_content_mean = math.exp(np.array(results_content['neg_log_likelihood']).mean())
    perp_titles_mean = math.exp(np.array(results_titles['neg_log_likelihood']).mean())
    
    results_regions[region].append({'content': results_content, 'titles': results_titles, 'perp_mean_content': perp_content_mean, 'perp_mean_titles': perp_titles_mean})

Ardeal


100%|██████████| 1542/1542 [05:07<00:00,  5.02it/s]


Banat


100%|██████████| 1124/1124 [03:41<00:00,  5.07it/s]


Bucovina


...

100%|██████████| 723/723 [02:23<00:00,  5.05it/s]


Ucraina


100%|██████████| 3010/3010 [10:00<00:00,  5.02it/s]


UK


100%|██████████| 499/499 [01:40<00:00,  4.95it/s]


In [12]:
with open("results_regions_qlora_ro.json", "w") as f:
    json.dump(results_regions, f, indent=4)